In [3]:
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
pip install gdown

In [ ]:
# pathGD = '1Eb1cyb5Ba9P6UPrc61kEchVSINFQgwSm'
# !gdown --id 1Eb1cyb5Ba9P6UPrc61kEchVSINFQgwSm

In [ ]:
pathTest = '1-39UmctH0WGWH11Rej51O8wl9Vkp0RLy'
pathTrain = '1-1prErSeosKCZydn7SKAccXNjEsBTAXE'
!gdown --id 1-1prErSeosKCZydn7SKAccXNjEsBTAXE


In [ ]:
!gdown --id 1-39UmctH0WGWH11Rej51O8wl9Vkp0RLy

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## Here we are specifically using EMNIST letter dataset. So Let's get started

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf


tags = ["plus", 'minus', 'equal', 'lparentheses', 'rparentheses', 'lbrackets', 'rbrackets',
        'X', 'y', 'z', 'A', 'b', 'C', 'd' ] + list(str(s) for s in range(0,10))

In [ ]:
x28 = [str(x) for x in range(784)]
headerList = ['label'] + x28


In [ ]:
len(tags)

In [ ]:

d_train = pd.read_pickle('/kaggle/working/data_train.pkl')
d_test = pd.read_pickle('/kaggle/working/data_test.pkl')


In [ ]:
d_train.head(10)

In [ ]:
d_train.info(memory_usage='deep')

In [ ]:
import gc
gc.collect()

In [ ]:
d_train.memory_usage(deep=True)* 1e-6

In [ ]:
print(d_train.memory_usage(deep=True)*1e-6)
print("total:", d_train.memory_usage(deep=True).sum()*1e-6)

In [ ]:
# import collections
# ts = [item for item, count in collections.Counter(df.loc[:, "label"].to_numpy()).items() if count > 1]
# print(len(ts))


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x1, x2, y1, y2 = [ d_train.loc[:, "0":].to_numpy(), d_test.loc[:, "0":].to_numpy(), 
                  d_train.loc[:,"label"].to_numpy().astype(str), d_test.loc[:,"label"].to_numpy().astype(str)]
y1

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencode = LabelEncoder()
y1, y2 = [labelencode.fit_transform(y1), labelencode.fit_transform(y2)]
y1, y2 = [tf.keras.utils.to_categorical(y1, len(tags)), tf.keras.utils.to_categorical(y2, len(tags))]

In [ ]:
Xtest = ((x1[:16] > .7).astype(int)*255).astype(int)

## Let's see what type of images we have got. 

In [ ]:
import matplotlib.pyplot as plt 
fig,axes = plt.subplots(3,5,figsize=(10,8))
for i,ax in enumerate(axes.flat):
    ax.imshow((Xtest[i].reshape([28,28])))


# Preprocesssing

# 1. Normalisation and reshaping of data

### Before feeding in the data to the model we will normalise and reshape the data given to us. This will decrease the complexity of the models and make the model work efficiently as less complex numbers will be there to process.

In [ ]:
# Normalise and reshape data
train_images = x1
test_images = x2 

train_images_number = train_images.shape[0]
train_images_height = 28
train_images_width = 28
train_images_size = train_images_height*train_images_width

train_images = train_images.reshape(train_images_number, train_images_height, train_images_width, 1)

test_images_number = test_images.shape[0]
test_images_height = 28
test_images_width = 28
test_images_size = test_images_height*test_images_width

test_images = test_images.reshape(test_images_number, test_images_height, test_images_width, 1)

# Lets create a CNN for the classification of these random images.

In [ ]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense,Dropout,MaxPooling2D,Conv2D
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.backend import clear_session
from keras.models import load_model

In [1]:
train_x,test_x,train_y,test_y = train_test_split(train_images,y1,test_size=0.2,random_state = 42)
np.shape(train_x[0])

In [ ]:
clear_session()
model = tf.keras.Sequential([ 
    tf.keras.layers.Conv2D(64,5,input_shape=(28,28,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,5,input_shape=(28,28,64), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64,3,input_shape=(14,14,32), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,3,input_shape=(14,14,64), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(input_shape=(7,7,64)),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(len(tags),activation='softmax')
])

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

## Adding Callback API's to save best weights and change lr

In [ ]:
 
MCP = ModelCheckpoint('Best_points.h5',verbose=1,save_best_only=True,monitor='val_accuracy',mode='max')
ES = EarlyStopping(verbose=1,patience=10,mode='min')
RLP = ReduceLROnPlateau(monitor='val_loss',patience=5,factor=0.2,min_lr=1e-5)

In [ ]:
TC = tf.keras.callbacks.TensorBoard("logs")
history = model.fit(train_x,train_y,epochs=120,validation_data=(test_x,test_y),callbacks=[MCP,ES,RLP, TC])

## Let's plot Accuracy vs Val_Accuracy to further evaluation..

In [ ]:
import seaborn as sns

In [ ]:
q = len(history.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+q),y = history.history['accuracy'], label='Accuracy')
sns.lineplot(x = range(1,1+q),y = history.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuray')

In [ ]:
train_x2,test_x2,train_y2,test_y2 = train_test_split(train_images,y1,test_size=0.15,random_state = 42)

In [ ]:
history1 = model.fit(train_x,train_y,epochs=120,validation_data=(test_x,test_y),callbacks=[MCP,ES,RLP, TC])

In [ ]:
q = len(history1.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+q),y = history1.history['accuracy'], label='Accuracy')
sns.lineplot(x = range(1,1+q),y = history1.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuray')

In [2]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label');

In [ ]:
model = load_model('/kaggle/input/best-point/Best_points.h5')


In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(test_images)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 2) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y2,axis = 1) 


In [ ]:
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = tags) 

In [ ]:
Y_true = np.argmax(y2,axis = 1) 

# Show some wrong results, and the difference between the predicted label and the real labe
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = test_images[errors]

dictY = d_test[["label"]].astype(str).copy()
dictY['code'] = labelencode.fit_transform(dictY['label'].values)
dictY = dictY.drop_duplicates('label')
dictY = dictY.to_numpy()

In [ ]:
dictY = dict(zip(dictY[:, 1], dictY[:, 0]))

In [ ]:
dictY

In [ ]:
def display_errors(errors_index,img_errors,pred_errors, obs_errors, encode):
    """ This function shows 6 images with their predicted and real labels"""

    for n in range(len(obs_errors)):
        error = errors_index[n]
        plt.imshow((img_errors[error]).reshape((28,28)))
        plt.title("Predicted label :{}\nTrue label :{}".format(encode[pred_errors[error]],encode[obs_errors[error]]))
        plt.show()

In [ ]:
# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors, dictY)